# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Li  ->  S. Li  |  ['S. Li']
L. Acuña  ->  L. Acuña  |  ['L. Acuña']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Arxiv has 69 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2411.09278


extracting tarball to tmp_2411.09278...

 done.
Retrieving document from  https://arxiv.org/e-print/2411.09506


extracting tarball to tmp_2411.09506...

...

✔ → 87318:\section{Summary and Conclusions}\label{sec:conclusions}
  ↳ 93583:\begin{appendix}
✔ → 93583:\begin{appendix}
  ↳ 93625:\section{Log of observations}\label{App:RVs}


✔ → 93625:\section{Log of observations}\label{App:RVs}
  ↳ 137845:\section{Planet model priors}
✔ → 137845:\section{Planet model priors}
  ↳ 140013:\section{Binary system model corner plot}\label{App:corner_binary}
✔ → 140013:\section{Binary system model corner plot}\label{App:corner_binary}
  ↳ 140357:\section{Planet model corner plot}\label{App:corner_planet}
✔ → 140357:\section{Planet model corner plot}\label{App:corner_planet}
  ↳ 140756:end


L. Acuña  ->  L. Acuña  |  ['L. Acuña']


Found 138 bibliographic references in tmp_2411.09506/aanda.bbl.
Issues with the citations
syntax error in line 403: '=' expected
Retrieving document from  https://arxiv.org/e-print/2411.09608


/tmp/ipykernel_2705/2822249172.py:52: LatexWarning: 2411.09608 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.09506-b31b1b.svg)](https://arxiv.org/abs/2411.09506) | **Gl 725A b: a potential super-Earth detected with SOPHIE and SPIRou in an M dwarf binary system at 3.5 pc**  |
|| P. Cortes-Zuleta, et al. -- incl., <mark>L. Acuña</mark> |
|*Appeared on*| *2024-11-15*|
|*Comments*| *26 pages, 17 figures. Accepted in A&A*|
|**Abstract**|            We report the discovery of a super-Earth candidate orbiting the nearby mid M dwarf Gl\,725A using the radial velocity (RV) method. The planetary signal has been independently identified using high-precision RVs from the SOPHIE and SPIRou spectrographs, in the optical and near-infrared domains, respectively. We modelled the stellar activity signal jointly with the planet using two Gaussian Processes, one for each instrument to account for the chromaticity of the stellar activity and instrumental systematics, along with a Keplerian model. The signal is significantly detected with a RV semi-amplitude of $1.67\pm0.20$ m/s. The planet Gl 725A b is found to be in an orbit compatible with circular with a period of $11.2201\pm0.0051$ days. We analysed 27 sectors of TESS photometry on which no transit event was found. We determined a minimum mass of $M_{p}\sin{i}=2.78\pm0.35\,M_{\oplus}$ which places the planet in the super-Earth regime. Using Mass-Radius relationships we predict a planetary radius to be between 1.2 and $2.0\,R_{\oplus}$. The proximity of Gl 725A, of only 3.5 pc, makes this new exoplanet one of the closest to Earth and joins the group of S-type low-mass planets in short orbits ($P<15$ d) around close M dwarfs.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.09278-b31b1b.svg)](https://arxiv.org/abs/2411.09278) | **A Recent Supermassive Black Hole Binary in the Galactic Center Unveiled by the Hypervelocity Stars**  |
|| C. Cao, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2024-11-15*|
|*Comments*| *34 pages, 13 figures, 1 table, submitted on 2024 January 15*|
|**Abstract**|            Dozens of B-type hypervelocity stars (HVSs) moving faster than the Galactic escape speed have been discovered in the Galactic halo and are produced most likely by the supermassive black hole (SMBH) at the Galactic Center (GC). However, the velocity distribution and in particular the deficit of the HVSs above 700 km/s is seriously inconsistent with the expectations of the present models. Here we show that the high-velocity deficit is due to the deficiency in close interactions of stars with the SMBH, because an orbiting intermediate-mass black hole (IMBH) of about 15,000 Solar mass kicked away slowly approaching stars 50-250 million years ago. The SMBH-IMBH binary formed probably after the merger of the Galaxy with the Gaia-Sausage-Enceladus (GSE) dwarf galaxy, and coalesced about 10 million years ago. Afterwards, HVSs with speed up to above 3000 km/s are produced by binary tidal disruptions and the counterparts formed the S-star cluster at the GC.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2411.09608-b31b1b.svg)](https://arxiv.org/abs/2411.09608) | **VST-SMASH: the VST Survey of Mass Assembly and Structural Hierarchy**  |
|| C. Tortora, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-11-15*|
|*Comments*| *4 pages, 2 figures, 1 table, published in the ESO Messenger 193*|
|**Abstract**|            The VLT Survey Telescope Survey of Mass Assembly and Structural Hierarchy (VST-SMASH) aims to detect tidal features and remnants around very nearby galaxies, a unique and essential diagnostic of the hierarchical nature of galaxy formation. Leveraging optimal sky conditions at ESO's Paranal Observatory, combined with the VST's multi-band optical filters, VST-SMASH aims to be the definitive survey of stellar streams and tidal remnants in the Local Volume, targeting a low surface-brightness limit of $\mu \sim$ 30 mag arcsec$^{-2}$ in the g and r bands, and $\mu \sim$ 28 mag arcsec$^{-2}$ in the i band, in a volume-limited sample of local galaxies within 11 Mpc and the Euclid footprint.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2411.09506/./figs/periodograms_freq.png', 'tmp_2411.09506/./figs/periodograms_original_corrected_Gl725A.png', 'tmp_2411.09506/./figs/Mdwarfs+Gl725Ab_mass_period_histograms_2024.png']
copying  tmp_2411.09506/./figs/periodograms_freq.png to _build/html/
copying  tmp_2411.09506/./figs/periodograms_original_corrected_Gl725A.png to _build/html/
copying  tmp_2411.09506/./figs/Mdwarfs+Gl725Ab_mass_period_histograms_2024.png to _build/html/
exported in  _build/html/2411.09506.md
    + _build/html/tmp_2411.09506/./figs/periodograms_freq.png
    + _build/html/tmp_2411.09506/./figs/periodograms_original_corrected_Gl725A.png
    + _build/html/tmp_2411.09506/./figs/Mdwarfs+Gl725Ab_mass_period_histograms_2024.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\DIFaddtex}[1]{{\bf #1}}$
$\newcommand{\DIFdeltex}[1]$
$\newcommand{\DIFaddbegin}$
$\newcommand{\DIFaddend}$
$\newcommand{\DIFdelbegin}$
$\newcommand{\DIFdelend}$
$\newcommand{\DIFmodbegin}$
$\newcommand{\DIFmodend}$
$\newcommand{\DIFaddFL}[1]{\DIFadd{#1}}$
$\newcommand{\DIFdelFL}[1]{\DIFdel{#1}}$
$\newcommand{\DIFaddbeginFL}$
$\newcommand{\DIFaddendFL}$
$\newcommand{\DIFdelbeginFL}$
$\newcommand{\DIFdelendFL}$
$\newcommand{\DIFadd}[1]{\texorpdfstring{\DIFaddtex{#1}}{#1}}$
$\newcommand{\DIFdel}[1]{\texorpdfstring{\DIFdeltex{#1}} }$
$\newcommand{\ha}{\rm H\alpha}$
$\newcommand{\ms}{m s^{-1}}$
$\newcommand{\teff}{T_{\rm eff}}$
$\newcommand{\logg}{\log g}$
$\newcommand{\vsini}{v\sin i}$
$\newcommand{\feh}{[Fe/H]}$
$\newcommand{\kms}{\hbox{km s^{-1}}}$
$\newcommand{\prot}{P_{\rm rot} = 103.1\pm6.1  \rm d }$
$\newcommand{\protv}{103.1\pm6.1 }$
$\newcommand{\starA}{\rm Gl~725A }$
$\newcommand{\DIFscaledelfig}{0.5}$
$\newcommand{\DIFaddincludegraphics}[2][]{{\color{blue}\fbox{\DIFOincludegraphics[#1]{#2}}}}$
$\newcommand{\DIFdelincludegraphics}[2][]{$
$\sbox{\DIFdelgraphicsbox}{\DIFOincludegraphics[#1]{#2}}$
$\settoboxwidth{\DIFdelgraphicswidth}{\DIFdelgraphicsbox}$
$\settoboxtotalheight{\DIFdelgraphicsheight}{\DIFdelgraphicsbox}$
$\scalebox{\DIFscaledelfig}{$
$\parbox[b]{\DIFdelgraphicswidth}{\usebox{\DIFdelgraphicsbox}\\[-\baselineskip] \rule{\DIFdelgraphicswidth}{0em}}\llap{\resizebox{\DIFdelgraphicswidth}{\DIFdelgraphicsheight}{$
$\setlength{\unitlength}{\DIFdelgraphicswidth}$
$\begin{picture}(1,1)$
$\thicklines\linethickness{2pt}{\color[rgb]{1,0,0}\put(0,0){\framebox(1,1) }}{\color[rgb]{1,0,0}\put(0,0){\line( 1,1){1}}}{\color[rgb]{1,0,0}\put(0,1){\line(1,-1){1}}}$
$\end{picture}$
$}\hspace*{3pt}}}$
$}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand\hyper{@linkstart##1##2 $
$     }$
$\newcommand\hyper{@linkstart##1##2 $
$     }$
$\newcommand\hyper{@linkstart##1##2 $
$     }$
$\newcommand\hyper{@linkstart##1##2 $
$     }$</div>



<div id="title">

# Gl 725A b: a potential super-Earth detected with SOPHIE and SPIRou in an M dwarf binary system at 3.5 pc

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2411.09506-b31b1b.svg)](https://arxiv.org/abs/2411.09506)<mark>Appeared on: 2024-11-15</mark> -  _26 pages, 17 figures. Accepted in A&A_

</div>
<div id="authors">

P. Cortés-Zuleta, et al. -- incl., <mark>L. Acuña</mark>

</div>
<div id="abstract">

**Abstract:** We report the discovery of a super-Earth candidate orbiting the nearby mid M dwarf Gl 725A using the radial velocity (RV) method. The planetary signal has been independently identified using high-precision RVs from the SOPHIE and SPIRou spectrographs, in the optical and near-infrared domains, respectively. We modelled the stellar activity signal jointly with the planet using two Gaussian Processes, one for each instrument to account for the chromaticity of the stellar activity and instrumental systematics, along with a Keplerian model. The signal is significantly detected with a RV semi-amplitude of $1.67\pm0.20$ m/s. The planet Gl 725A b is found to be in an orbit compatible with circular with a period of $11.2201\pm0.0051$ days. We analysed 27 sectors of TESS photometry on which no transit event was found. We determined a minimum mass of $M_{p}\sin{i}=2.78\pm0.35 M_{\oplus}$ which places the planet in the super-Earth regime. Using Mass-Radius relationships we predict a planetary radius to be between 1.2 and $2.0 R_{\oplus}$ . The proximity of Gl 725A, of only 3.5 pc, makes this new exoplanet one of the closest to Earth and joins the group of S-type low-mass planets in short orbits ( $P<15$ d) around close M dwarfs.

</div>

<div id="div_fig1">

<img src="tmp_2411.09506/./figs/periodograms_freq.png" alt="Fig1" width="100%"/>

**Figure 1. -** Periodograms of the SOPHIE and SPIRou RVs and activity indicators. The pointed and dashed horizontal lines mark the 0.1 and 1$\%$ FAP. The magenta and orange dashed vertical lines depict the orbital period of the planet at 11.2 d and the stellar rotation period at 103.1 d, respectively. (*fig:periodograms*)

</div>
<div id="div_fig2">

<img src="tmp_2411.09506/./figs/periodograms_original_corrected_Gl725A.png" alt="Fig3" width="100%"/>

**Figure 3. -** Periodograms of the original SPIRou RVs (top) and after the \texttt{wapiti} correction (bottom), on which the period with the highest power correspond to the planet signal at 11.2 days. The horizontal dashed lines in blue, orange, and green depict the FAP at 10, 1, and 0.1$\%$, respectively. The vertical line indicates the stellar rotation period at 103.1 days. Details odf the \texttt{wapiti} correction can be found in Sect. \ref{wapiti}. (*fig:wapiti_periodograms*)

</div>
<div id="div_fig3">

<img src="tmp_2411.09506/./figs/Mdwarfs+Gl725Ab_mass_period_histograms_2024.png" alt="Fig7" width="100%"/>

**Figure 7. -** Orbital period versus mass diagram of planets around M dwarfs. For simplicity, planets with orbits larger than $10^5$ days are not included. The orange contours indicate 10, 30, 60, and 90$\%$ of density levels of the observed probability density of the joint mass and period distribution. The magenta star and lines indicate the position of Gl 725A b, with a minimum mass of $M{_p}\sin{i}=2.78\pm0.35 M_{\oplus}$, and an orbital period of 11.2 days. The top and right histograms correspond to the probability density distribution of the orbital periods and planetary masses, respectively. For reference, the black diamonds represent the Earth and Neptune. (*fig:Mdwarf_planets*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2411.09506"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

246  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
